In [14]:
import numpy as np
from cities.utils.data_grabber import find_repo_root, DataGrabber
import pandas as pd
import numpy as np
import requests

data = DataGrabber()
data.get_features_wide(["gdp"])
gdp = data.wide["gdp"]

root = find_repo_root()

In [5]:
area = 'county:*&in=state:*' 

variables = "DP04_0045E,DP04_0046E,DP04_0089E,DP04_0134E"


url = f'https://api.census.gov/data/2022/acs/acs5/profile?get={variables}&for={area}'

response = requests.get(url)

assert response.status_code == 200 # 200 means success

data = response.json()

df = pd.DataFrame(data[1:], columns=data[0])
df.columns


Index(['DP04_0045E', 'DP04_0046E', 'DP04_0089E', 'DP04_0134E', 'state',
       'county'],
      dtype='object')

In [22]:
homeownership = df.copy()

column_mapping = {
    'DP04_0045E': 'occupied_housing_units',
    'DP04_0046E': 'owner_occupied',
    'DP04_0089E': 'median_owner_occupied_home_value',
    'DP04_0134E': 'median_rent'
    
}
homeownership.rename(columns=column_mapping, inplace=True)

homeownership['GeoFIPS'] = homeownership['state'] + homeownership['county']
homeownership.drop(['state', 'county'], axis=1, inplace=True)
homeownership['GeoFIPS'] = homeownership['GeoFIPS'].astype(np.int64)

homeownership.head()

,occupied_housing_units,owner_occupied,median_owner_occupied_home_value,median_rent,GeoFIPS
0,22308,16832,191800,1199,1001
1,90802,70708,266000,1160,1003
2,9016,5858,102700,627,1005
3,7216,5465,120100,765,1007
4,21626,17144,159800,734,1009


In [23]:
homeownership['homeownership_rate'] = (homeownership['owner_occupied'].astype(float) / homeownership['occupied_housing_units'].astype(float)) * 100

homeownership.drop(['owner_occupied', 'occupied_housing_units'], axis=1, inplace=True)


In [26]:
common_fips = np.intersect1d(homeownership['GeoFIPS'].unique(), gdp['GeoFIPS'].unique())
homeownership = homeownership[homeownership['GeoFIPS'].isin(common_fips)]
homeownership = homeownership.merge(gdp[['GeoFIPS', 'GeoName']], on='GeoFIPS', how='left')
homeownership = homeownership.reindex(columns=['GeoFIPS', 'GeoName'] + list(homeownership.columns.drop(['GeoFIPS', 'GeoName'])))

homeownership.iloc[:, 2:] = homeownership.iloc[:, 2:].astype(float)

homeownership.head()


,GeoFIPS,GeoName,median_owner_occupied_home_value,median_rent,homeownership_rate
0,1001,"Autauga, AL",191800.0,1199.0,75.452752
1,1003,"Baldwin, AL",266000.0,1160.0,77.870531
2,1005,"Barbour, AL",102700.0,627.0,64.973381
3,1007,"Bibb, AL",120100.0,765.0,75.734479
4,1009,"Blount, AL",159800.0,734.0,79.274947


In [28]:
variables = ['median_owner_occupied_home_value', 'median_rent', 'homeownership_rate']

for variable in variables:
    hazard_variable = homeownership[["GeoFIPS", "GeoName", variable]]
    hazard_variable.to_csv(f"{root}/data/raw/{variable}.csv", index=False)
